In [31]:
import os
import pandas as pd
import numpy as np
import urllib
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

In [32]:
path = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\normals\tmean_regression'
csv = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\normals\tmean_regression\csv.csv'
xl_file = r"C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\colocation_metadata.xlsx"
f = open(csv,'w+')
os.chdir(path)

In [33]:
xl = pd.ExcelFile(xl_file)

meta = xl.parse('metadata')
meta['sdate'] = meta.sdate.dt.date
meta['edate'] = meta.edate.dt.date
meta = meta.drop(meta.columns[3],axis=1)

In [36]:
n = 0
for index,rows in meta.iterrows():
    try:
        print('Grabbing temperature data for ' + str(meta.ID[n]))
        urllib.request.urlretrieve(
            'http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=daily&sids=' + str(meta.ID[n]) + '&sdate=' + str(meta.sdate[n]) + '&edate=' + str(meta.edate[n]) + '&elems=tave,tmax,tmin',
            filename=csv)
        df = pd.read_csv(csv)
        df = df.reset_index()
    except Exception as e:
        print('Something went wrong while grabbing temperature data for ' + str(meta.ID[n]) + ': ' + e.args[0])
    
    try:
        print('Cleaning temperature data for ' + str(meta.ID[n]))
        df.columns.values[0] = 'station'
        df.columns.values[1] = 'date'
        df.columns.values[2] = 'tmean'
        df.columns.values[3] = 'tmax'
        df.columns.values[4] = 'tmin'
        df['date'] = pd.to_datetime(df.date)
        df['julian'] = df.date.dt.strftime('%j')
        
        df.tmean = df.tmean.astype(float)
        df.tmax = df.tmax.astype(float)
        df.tmin = df.tmin.astype(float)
        
        df.tmean = 32 + (df.tmean * (9/5))
        df.tmax = 32 + (df.tmax * (9/5))
        df.tmin = 32 + (df.tmin * (9/5))
        df = df.dropna()
    except Exception as e:
        print('Something went wrong while cleaning the temperature data for ' + str(meta.ID[n]) + ': ' + e.args[0])
    
    try:
        print('Calculating temperature normals for ' + str(meta.ID[n]))
        norms = df.groupby(['julian']).mean().reset_index()
        norms = norms[:-1]
    except Exception as e:
        print('Something went wrong while calculating the temeprature normals for ' + str(meta.ID[n]))

    try:
        print('Fitting a curve equation to daily tmean normals for ' + str(meta.ID[n]))
        t = norms['julian'].values
        tmean = norms['tmean'].values
        c,cov = curve_fit(lob,t,tmean)
    except Exception as e:
        print('Something went wrong whlie fitting a curve equation to the daily tmean normals for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Populating a dataset with values derived from the curve equation for ' + str(meta.ID[n]))
        n = len(df['julian'])
        y = np.empty(n)
        def lob (x,a,b,c,d,e,f):
            return(a*x**5) + (b*x**4) + (c*x**3) + (d*x**2) + (e*x) + f
        for i in range(n):
            y[i] = lob(norms['julian'][i],c[0],c[1],c[2],c[3],c[4],c[5])
    except Exception as e:
        print('Something went wrong while populating a dataset using the curve equation for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Calculating the coefficient of determination for ' + str(meta.ID[n]))
        score = r2_score(df['tmean'],y)
        print('Coefficient of determination for the curve created from ' + str(meta.ID[n]) + 'was: ' + str(score))
    except Exception as e:
        print('Soemthing went wrong while calculating the coefficent of determination for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Merging the curve dataset for ' + str(meta.ID[n]) + ' with a julian day index')
        j_day = df['julian']
        j_day = j_day.reset_index()
        series = pd.Series(y)
        df2 = series.reset_index()
        norms = df2.merge(j_day)
        norms.columns.values[1] = 'tmean_normal'
    except Exception as e:
        print('Something went wrong while merging the curve data set for ' + str(meta.ID[n]) + ' with a julian day index: ' + e.args[0])
    try:
        print('Storing the daily average temperature normals for ' + str(meta.ID[n]))
        norms.to_csv(path + '\CoAg_tmean_curve_' + str(meta.ID[n]) + '.csv')
    except Exception as e:
        print('Something went wrong while storing the daily average temperature normals for ' + str(meta.ID[n]) + ': ' + e.args[0])
    n=n+1
print('tmean normal datasets successfully created!')

Grabbing temperature data for AKR02
Cleaning temperature data for AKR02
Calculating temperature normals for AKR02
Fitting a curve equation to daily tmean normals for AKR02
Populating a dataset with values derived from the curve equation for AKR02


KeyError: 2359